In [45]:
from pipeline.datasets.sevir.sevir import SEVIRLightningDataModule
import time
from omegaconf import OmegaConf
cfg = OmegaConf.load("/home/vatsal/NWM/weatherforecasting/pipeline/datasets/sevir/config.yaml").Dataset
dm = SEVIRLightningDataModule(cfg)
dm.prepare_data()
setup_s = time.time()
dm.setup()
setup_e = time.time()
print(f"took : {setup_e - setup_s}")

l_s = time.time()
loader = dm.train_dataloader()
idx = 0
for sample in loader:
    idx += 1
    if idx >= 200:break
l_e = time.time()
print("loading", l_e - l_s)
l_train_full = len(dm.train_dataloader())
l_val_full = len(dm.val_dataloader())
l_test_full = len(dm.test_dataloader())
print(l_train_full, l_val_full, l_test_full)


Seed set to 0


took : 15.291179895401001
loading 109.07367300987244
5038 560 1520


In [46]:
from pipeline.datasets.sevir.sevir import SEVIRLightningDataModule
import time
from omegaconf import OmegaConf
cfg = OmegaConf.load("/home/vatsal/NWM/weatherforecasting/pipeline/datasets/sevir/fast_config.yaml").Dataset
dm = SEVIRLightningDataModule(cfg)
dm.prepare_data()
setup_s = time.time()
dm.setup()
setup_e = time.time()
print(f"took : {setup_e - setup_s}")

l_s = time.time()
loader = dm.train_dataloader()
idx = 0
for sample in loader:
    idx += 1
    if idx >= 200:break
l_e = time.time()
print("loading", l_e - l_s)
l_train_fast = len(dm.train_dataloader())
l_val_fast = len(dm.val_dataloader())
l_test_fast = len(dm.test_dataloader())
print(l_train_fast, l_val_fast, l_test_fast)

Seed set to 0


took : 2.798424005508423
loading 14.54524302482605
916 102 236


In [41]:
print(l_train_fast/l_train_full)
print(l_val_fast/l_val_full)
print(l_test_fast/l_test_full)

0.18181818181818182
0.18214285714285713
0.15526315789473685


In [ ]:
dummpy = torch.randn(8, 52, 2304)
from omegaconf import OmegaConf
cfg = OmegaConf.load("/home/vatsal/NWM/weatherforecasting/experiments/pretrained_ae_dlinear_indc_indp/config.yaml").dlinear
model = DLinear(cfg)
model2 = DLinear2(cfg)
model.train()
model2.train()
state_dict = model.state_dict()
model2.load_state_dict(state_dict)
out1 = model(dummpy)
out2 = model2(dummpy)
#check all parameters are same
flag = True
for p1, p2 in zip(model.parameters(), model2.parameters()):
    if not torch.allclose(p1, p2, atol=1e-4):
        flag = False
        break
print("Parameters match:", flag)
print(out1.shape, out2.shape)
print(torch.allclose(out1, out2, atol=1e-12))  #
target = torch.randn(8, 48, 2304) 
loss1 = F.mse_loss(out1, target)
loss2 = F.mse_loss(out2, target)

loss1.backward()
loss2.backward()
optim = torch.optim.SGD(model.parameters(), lr=1e-3)
optim2 = torch.optim.SGD(model2.parameters(), lr=1e-3)
optim.step()
optim2.step()

# Check if the gradients are the same
flag = True
for (name1, p1), (name2, p2) in zip(model.named_parameters(), model2.named_parameters()):
    if p1.grad is None or p2.grad is None:
        print(f"Gradient is None for: {name1}, {name2}")
        assert p1.grad is None and p2.grad is None, "both gradients should be None"
        print("skipping gradient check for None gradients")
        continue
    if not torch.allclose(p1.grad, p2.grad, atol=1e-4):
        flag = False
        break
print("Gradients match:", flag)

out1_again = model(dummpy)
out2_again = model2(dummpy)
print(torch.allclose(out1_again, out2_again, atol=1e-12))  # should be True after training

Parameters match: True
torch.Size([8, 48, 2304]) torch.Size([8, 48, 2304])
True
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient check for None gradients
skipping gradient

In [3]:
import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_ch)
        self.act1 = nn.GELU()
        
        self.conv2 = nn.Conv2d(out_ch, out_ch, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_ch)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_ch != out_ch:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_ch)
            )
            
        self.act2 = nn.GELU()

    def forward(self, x):
        shortcut = self.shortcut(x)
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.act1(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        out += shortcut
        return self.act2(out)

class UpsampleBlock(nn.Module):
    def __init__(self, in_ch, out_ch, scale_factor):
        super().__init__()
        self.upsample = nn.Upsample(scale_factor=scale_factor, mode='nearest')
        self.resblock = ResidualBlock(in_ch, out_ch, stride=1)

    def forward(self, x):
        return self.resblock(self.upsample(x))

class ConvAutoencoder(nn.Module):
    def __init__(self, in_ch=1, latent_dim=512):
        super().__init__()
        
        # Encoder: 128 → 64 → 32 → 16 → 4 → 1
        self.enc1 = ResidualBlock(in_ch,   64, stride=2)  # 128 → 64
        self.enc2 = ResidualBlock(64,     128, stride=2)  # 64 → 32
        self.enc3 = ResidualBlock(128,    256, stride=2)  # 32 → 16
        self.enc4 = ResidualBlock(256,    512, stride=4)  # 16 → 4
        self.enc5 = ResidualBlock(512,   1024, stride=4)  # 4 → 1

        self.flatten = nn.Flatten()                     # (B,1024,1,1) → (B,1024)
        self.fc_enc = nn.Linear(1024, latent_dim)

        self.fc_dec = nn.Linear(latent_dim, 1024)
        self.unflatten = nn.Unflatten(1, (1024, 1, 1))   # (B,1024) → (B,1024,1,1)
        self.dec_init_conv = ResidualBlock(1024, 1024, stride=1)

        # Decoder: 1 → 4 → 16 → 32 → 64 → 128
        self.dec1 = UpsampleBlock(1024, 512, scale_factor=4)  # 1 → 4
        self.dec2 = UpsampleBlock(512,  256, scale_factor=4)  # 4 → 16
        self.dec3 = UpsampleBlock(256,  128, scale_factor=2)  # 16 → 32
        self.dec4 = UpsampleBlock(128,   64, scale_factor=2)  # 32 → 64
        
        self.final_upsample = nn.Upsample(scale_factor=2, mode='nearest') # 64 → 128
        self.final_conv = nn.Conv2d(64, in_ch, kernel_size=3, stride=1, padding=1)

    def encode(self, x):
        x = self.enc1(x)
        x = self.enc2(x)
        x = self.enc3(x)
        x = self.enc4(x)
        x = self.enc5(x)
        x = self.flatten(x)
        z = self.fc_enc(x)
        return z

    def decode(self, z):
        x = self.fc_dec(z)
        x = self.unflatten(x)
        x = self.dec_init_conv(x)
        x = self.dec1(x)
        x = self.dec2(x)
        x = self.dec3(x)
        x = self.dec4(x)
        x = self.final_upsample(x)
        x = self.final_conv(x)
        return x

    def forward(self, x):
        z = self.encode(x)
        reconstruction = self.decode(z)
        return reconstruction, z

if __name__ == "__main__":
    import torch
    model = ConvAutoencoder()
    x = torch.randn(8, 1, 128, 128)  # Batch of 8 images with 1 channel, 128x128 size
    recon, z = model(x)
    print("Input shape:", x.shape)
    print("Reconstruction shape:", recon.shape)
    print("Latent vector shape:", z.shape)
    params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    params_in_million = params / 1e6
    print(f"Total trainable parameters: {params_in_million:.2f} million")

Input shape: torch.Size([8, 1, 128, 128])
Reconstruction shape: torch.Size([8, 1, 128, 128])
Latent vector shape: torch.Size([8, 512])
Total trainable parameters: 49.58 million


In [1]:
path = "/home/vatsal/NWM/sevir_lr/data/vil/2017/SEVIR_VIL_RANDOMEVENTS_2017_0501_0831.h5"
import h5py
data = h5py.File(path, 'r')
vil = data['vil'][:]
print("Shape of VIL data:", vil.shape)

Shape of VIL data: (1440, 128, 128, 25)


In [1]:
import datetime
import os
os.environ['WANDB_API_KEY'] = '041eda3850f131617ee1d1c9714e6230c6ac4772'
datetime.datetime.now().strftime("_%Y%m%d_%H%M%S")
from pytorch_lightning.loggers import WandbLogger
exp_name = "test_run_xyz"
save_dir = os.path.join("experiments", "test", exp_name + datetime.datetime.now().strftime("_%Y%m%d_%H%M%S"))
version = "v1.0"
logger = WandbLogger(project="delete_it", name="test_run", save_dir=save_dir, version=version)

/home/vatsal/miniconda3/envs/earthformer/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
/home/vatsal/miniconda3/envs/earthformer/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
from pipeline.datasets.sevire.sevir import SEVIRLightningDataModule
data = SEVIRLightningDataModule(dataset_name="sevir_lr", num_workers=0, batch_size=1, seq_len=1, stride=1, layout='NTHW')
data.setup()
data.prepare_data()
train_loader = data.train_dataloader()
sample1 = next(iter(train_loader))

/home/vatsal/miniconda3/envs/earthformer/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
/home/vatsal/miniconda3/envs/earthformer/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
len(train_loader)

297650

In [1]:
from pipeline.datasets.sevire.sevir import SEVIRLightningDataModule
data2 = SEVIRLightningDataModule(dataset_name="sevir_lr", num_workers=0, batch_size=8, seq_len=1, stride=1, layout='NTHW')
data2.setup()
data2.prepare_data()
train_loader2 = data2.train_dataloader()
sample2 = next(iter(train_loader2))

/home/vatsal/miniconda3/envs/earthformer/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
/home/vatsal/miniconda3/envs/earthformer/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
len(train_loader2)

37206

In [3]:
from pipeline.datasets.sevire.sevir import SEVIRLightningDataModule
data3 = SEVIRLightningDataModule(dataset_name="sevir_lr", num_workers=0, batch_size=8, seq_len=1, stride=1, layout='NTHW')
data3.setup()
data3.prepare_data()
train_loader3 = data2.train_dataloader()

In [5]:
sample3 = next(iter(train_loader3))
import torch
torch.allclose(sample2['vil'], sample3['vil'], atol=1e-12) 

True

In [7]:
datas = []
import torch
for sample in train_loader2:
    data = sample['vil']
    datas.append(data)
print(len(datas))
for sample in train_loader3:
    data = sample['vil']
    idx = -1
    for i, d in enumerate(datas):
        if torch.allclose(data, d, atol=1e-12):
            idx = i
            break
    if idx != -1:
        print(f"Found matching data at index {idx}")
    else:
        print("No matching data found _______________________________-")
        break

34
Found matching data at index 0
Found matching data at index 1
Found matching data at index 2
Found matching data at index 3
Found matching data at index 4
Found matching data at index 5
Found matching data at index 6
Found matching data at index 7
Found matching data at index 8
Found matching data at index 9
Found matching data at index 10
Found matching data at index 11
Found matching data at index 12
Found matching data at index 13
Found matching data at index 14
Found matching data at index 15
Found matching data at index 16
Found matching data at index 17
Found matching data at index 18
Found matching data at index 19
Found matching data at index 20
Found matching data at index 21
Found matching data at index 22
Found matching data at index 23
Found matching data at index 24
Found matching data at index 25
Found matching data at index 26
Found matching data at index 27
Found matching data at index 28
Found matching data at index 29
Found matching data at index 30
Found matching 

In [9]:
datas = []
import torch
for sample in train_loader2:
    data = sample['vil']
    datas.append(data)
print(len(datas))
for sample in train_loader2:
    data = sample['vil']
    idx = -1
    for i, d in enumerate(datas):
        if torch.allclose(data, d, atol=1e-12):
            idx = i
            break
    if idx != -1:
        print(f"Found matching data at index {idx}")
    else:
        print("No matching data found _______________________________-")
        break

34
Found matching data at index 0
Found matching data at index 1
Found matching data at index 2
Found matching data at index 3
Found matching data at index 4
Found matching data at index 5
Found matching data at index 6
Found matching data at index 7
Found matching data at index 8
Found matching data at index 9
Found matching data at index 10
Found matching data at index 11
Found matching data at index 12
Found matching data at index 13
Found matching data at index 14
Found matching data at index 15
Found matching data at index 16
Found matching data at index 17
Found matching data at index 18
Found matching data at index 19
Found matching data at index 20
Found matching data at index 21
Found matching data at index 22
Found matching data at index 23
Found matching data at index 24
Found matching data at index 25
Found matching data at index 26
Found matching data at index 27
Found matching data at index 28
Found matching data at index 29
Found matching data at index 30
Found matching 

In [ ]:
datas = []
import torch
for sample in train_loader2:
    data = sample['vil']
    for bdata in data:
        datas.append(bdata)
print(len(datas))
for sample in train_loader3:
    data = sample['vil']
    for bdata in data:
        idx = -1
        for i, d in enumerate(datas):
            if torch.allclose(bdata, d, atol=1e-12):
                idx = i
                break
        if idx != -1:
            print(f"Found matching data at index {idx}")
        else:
            print("No matching data found _______________________________-")
            break